In [ ]:
## https://dnd.su/bestiary/

# Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd

/tmp/ipykernel_7190/2561702948.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import re

In [3]:
import time

# Parsing

In [31]:
response_text = requests.get('https://dnd.su/bestiary/').text
soup = BeautifulSoup(response_text, 'html.parser')
hrefs = soup.find_all('a', {'class': 'list-item-wrapper'})
href_list = []
for href in hrefs:
    href_list.append('https://dnd.su/' + str(href.get('href')))

print(href_list[0], href_list[-1])

https://dnd.su//bestiary/7758-alastrah/ https://dnd.su//bestiary/3659-yan_c_bin/


In [13]:
len(href_list)

2678

In [44]:
time_s = time.time()
response_card = requests.get('https://dnd.su/bestiary/7188-doomwake_giant/')
print(response_card.status_code)
response_card.headers["content-type"].strip().startswith("application/json") #Поплачем.
soup = BeautifulSoup(response_card.text, 'html.parser')
title = soup.find('div', {'class': 'cards-wrapper'}).find('h2').text.split(' ')[0]
print(time.time() - time_s)

200
0.15401840209960938


In [76]:
desc_all = soup.find_all('li', {'class':'subsection desc'})
for i in desc_all:
    if 'Описание' in str(i):
        print((' ').join(list(x.text for x in i.find_all('p'))))
        # print(i.find_all('p'))

Вера — это мощная сила в Теросе, способная творить чудеса и изменять мир, но не всегда к лучшему. Великаны надвигающегося рока возникают из пагубных страхов, которые заражают целый город или регион. Если достаточное количество смертных достаточно долго будут бояться разрушения, их ужас может проявиться в виде одного из этих огромных существ, порождённых Никсом. Оказавшись на свободе, великан надвигающегося рока приходит в ярость, исполняя пророчество, в которое верили смертные. После этого они обычно бродят по краям мира, где со временем либо исчезают, либо становятся легендами, живущими своей собственной ужасной жизнью. Целый спектр великанов претендует на территории по всему Теросу, черпая силу из аспектов самого мира — от древнего камня и бушующего пламени до глубин морей и небес. В отличие от многих существ из легенд, большинство великанов обязаны своим существованием не Никсу и мечтам смертных, а природным силам земли. В результате теросские великаны обычно наделены элементами, ко

In [104]:
data_list = []
error_list = []

def collect_data(url):
    try:
        resp = requests.get(url, timeout=60)

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'html.parser')
            try:
                title = soup.find('div', {'class': 'cards-wrapper'}).find('h2').text.split(' [')[0]
            except:
                title = url
                print(url)
                error_list.append(url)
            desc_all = soup.find_all('li', {'class':'subsection desc'})
            descr = []
            for i in desc_all:
                if 'Описание' in str(i):
                    descr.append((' ').join(list(x.text for x in i.find_all('p') if len(x) > 0)))
            new_data = pd.DataFrame({'name':[title], 'description':[(' ').join(descr)], 'url':[url]})
            
            data_list.append(new_data)
            print(f'Обработал страницу {title}')
        else:
            error_list.append(url)
            print(f'resp error. url = {url}')
            
    except:
        error_list.append(url)

In [100]:
response_card = requests.get('https://dnd.su//bestiary/946-izek_strazni/')
soup = BeautifulSoup(response_card.text, 'html.parser')
soup.find('div', {'class': 'cards-wrapper'}).find('h2').text.split(' ')[0]

'Айзек'

In [105]:
for i, href in enumerate(href_list):
    collect_data(href)

Обработал страницу Аластра
Обработал страницу Альмираж 
Обработал страницу Арабэлль
Обработал страницу Ааракокра симулякр
Обработал страницу Ааракокра
Обработал страницу Анарх
Обработал страницу Агата Серебряная ложка
Обработал страницу Амфисбена
Обработал страницу Амфисбена
Обработал страницу Аазон Талиери
Обработал страницу Алдани (Лобстеролюд)
Обработал страницу Альсеид
Обработал страницу Архелон
Обработал страницу Архелон зомби
Обработал страницу Астральная зараза
Обработал страницу Аартук звездный ужас
Обработал страницу Аартук травянник
Обработал страницу Ави
Обработал страницу Автогном
Обработал страницу Агдон Длинный шарф
Обработал страницу Акулотелое отродье
Обработал страницу Аллозавр
Обработал страницу Аллозавр зомби
Обработал страницу Алоизия Телфан
Обработал страницу Андер
Обработал страницу Аниматронный аллозавр
Обработал страницу Анхег
Обработал страницу Ардра Желчь
Обработал страницу Арло Киттлтоу (1-4 уровня)
Обработал страницу Архонт-светоч
Обработал страницу Ауспиция

In [110]:
for href in error_list:
    collect_data(href)

Обработал страницу Белаширра
Обработал страницу Доктор Дэннелл
Обработал страницу Зот
Обработал страницу Заблудший угнетённый
Обработал страницу Зимний эладрин
Обработал страницу Капитан Гармониума
Обработал страницу Король Перьев
Обработал страницу Крессаро
Обработал страницу Носящий пурпур Неронвейн
Обработал страницу Павлин
Обработал страницу Побрякушка Джангл


In [112]:
dnd_su_data = pd.concat(data_list)
dnd_su_data.to_pickle('dnd_su.xz')

In [6]:
dnd_su_data = pd.read_pickle('/home/darya/Документы/Programm_things/ipunb/dnd_su.xz')
dnd_su_data

,name,description
1,Альмираж,"Альмираж – большой, робкий кролик с закрученны..."
2,Арабэлль,Арабэлль имеет алебастрово-белую кожу и волосы...
3,Ааракокра симулякр,"Ааракокры населяют Воющий Вихрь, бесконечный ш..."
4,Амфисбена,У амфисбены есть голова на обоих концах её зме...
5,Алдани (Лобстеролюд),Многие Чульты могут вспомнить детские сказки о...
...,...,...
1915,Заблудший угнетённый,Царство Теней ведёт странников по кругу до той...
1916,Зимний эладрин,Эладрины пребывают в зелёном великолепии Стран...
1917,Капитан Гармониума,Капитан Гармониума возглавляют патрули миротво...
1918,Король Перьев,"Чудовищный тираннозавр рекс , известный как Ко..."


In [7]:
dnd_su_data = dnd_su_data.drop_duplicates(subset='description').reset_index(drop=True)

In [9]:
dnd_su_data = dnd_su_data.loc[dnd_su_data.description.str.len() > 10]

In [10]:
list(dnd_su_data.sample(5).description)

['Часто называемые платиновыми рыцарями, драконорожденные чемпионы Багамута могут принадлежать к ордену, который существует для защиты мира от зла — и особенно от злых драконов. Чемпион Багамута обладает характерной парой платиновых крыльев и оружием дыхания, наполненным энергией излучения.  Драконорождённый чемпионы. Связь между драконорожденными и их драконьими предками проявляется по-разному. Некоторые драконорожденные отождествляют себя с определенным видом драконов и пытаются подражать поступкам и поведению таких драконов. Другие считают свое драконье наследие — цветное, металлическое или самоцветное — чем-то вроде большой расширенной семьи. Но для чемпионов-драконорожденных эта связь является как духовной, так и биологической, и они посвящают себя своему божественному предку. Драконорожденные чемпионы продвигают дело своего бога-дракона среди драконьих существ и других народов.',
 "Оперативники - это агенты организаций, которые занимаются шпионажем. Лучшие из них часто рассматрив

In [129]:
html.unescape('Монахи Священного Камня изучают боевое ис\xadкусство, в основу которого положена философия')

'Монахи Священного Камня изучают боевое ис\xadкусство, в основу которого положена философия'

In [11]:
patt_bracketrd = re.compile(r'\([^()]*\)') #Убирает всё, что в круглых скобках "()".
patt_bracketsq = re.compile(r'\[[^\[\]]*\]') #Убирает всё, что в квадратных скобках "[]".
patt_list = [patt_bracketrd, patt_bracketsq]
for patt in patt_list:
    dnd_su_data.description = dnd_su_data.description.str.replace(patt, '', regex=True)

In [12]:
dnd_su_data.description = dnd_su_data.description.apply(lambda x: x.replace('\n', '').replace('\r', '').replace('\xad', ''))
dnd_su_data.description.loc[dnd_su_data.description.str.contains('\n')]

Series([], Name: description, dtype: object)

In [ ]:
# Убрать см. статью, см. дополнительно, смотри статью
# Убрать всё в скобках

In [18]:
stop_words = ['статью', 'бестиарий', 'см.']

def check_me(texts):
    temp_list = []
    texts = re.findall('[^.]+\.', texts)
    for text in texts:
        text = text.lower()
        if len(text) > 4:
            if len(text.split()) > 1:
                if not any(word in text for word in stop_words): 
                            temp_list.append(text)
    return ''.join(temp_list)

dnd_su_data.description = dnd_su_data.description.apply(lambda x: check_me(x)) 

In [36]:
list(dnd_su_data.loc[dnd_su_data.description.str.contains('дополнительно', regex=False)].description)

['тщеславная хозяйка дома кассалантер знает толк в мистических искусствах. как и её муж викторо, она поклоняется асмодею. в молодости амалия и викторо подписали договор с архидьяволом, продав души своих детей за власть, здоровье и долголетие. душу освальдо, их старшего сына, забрали немедленно, и он превратился в цепного дьявола. души их младших детей, близнецов теренцио и эльзерины, заберут, когда им исполнится девять лет. дополнительное условие договора позволяет кассалантерам выкупить их, но это требует колоссальной суммы денег и массового жертвоприношения несчастных. пока викторо рыщет в поисках клада дагульта неверембера, амалия готовится провести пир в честь дня основателей, на котором гостей отравят. амалия хорошо воспитана, начитанна, много путешествовала и крайне хитра. говорят, что с ней трудно сторговаться. она увлекается изучением бабочек и устроила у себя в поместье прекраснейший сад с бабочками. младшим детям она позволяет играть в этом саду под её присмотром.',
 'алчност

In [21]:
dnd_su_data

,name,description
0,Альмираж,"альмираж – большой, робкий кролик с закрученны..."
1,Арабэлль,арабэлль имеет алебастрово-белую кожу и волосы...
2,Ааракокра симулякр,"ааракокры населяют воющий вихрь, бесконечный ш..."
3,Амфисбена,у амфисбены есть голова на обоих концах её зме...
4,Алдани (Лобстеролюд),многие чульты могут вспомнить детские сказки о...
...,...,...
1906,Заблудший угнетённый,царство теней ведёт странников по кругу до той...
1907,Зимний эладрин,эладрины пребывают в зелёном великолепии стран...
1908,Капитан Гармониума,капитан гармониума возглавляют патрули миротво...
1909,Король Перьев,"чудовищный тираннозавр рекс , известный как ко..."


In [37]:
dnd_su_data = dnd_su_data.drop_duplicates(subset='description').reset_index(drop=True)
dnd_su_data

,name,description
0,Альмираж,"альмираж – большой, робкий кролик с закрученны..."
1,Арабэлль,арабэлль имеет алебастрово-белую кожу и волосы...
2,Ааракокра симулякр,"ааракокры населяют воющий вихрь, бесконечный ш..."
3,Амфисбена,у амфисбены есть голова на обоих концах её зме...
4,Алдани (Лобстеролюд),многие чульты могут вспомнить детские сказки о...
...,...,...
1872,Заблудший угнетённый,царство теней ведёт странников по кругу до той...
1873,Зимний эладрин,эладрины пребывают в зелёном великолепии стран...
1874,Капитан Гармониума,капитан гармониума возглавляют патрули миротво...
1875,Король Перьев,"чудовищный тираннозавр рекс , известный как ко..."


In [39]:
dnd_su_data[['name', 'description']].to_pickle('datas/dnd_su.xz')

In [ ]:
'''
Сделано:
- Убрала 'см. дополнительно статью' и подобное по тегам ['статью', 'бестиарий', 'см.']
- Убрала всё, что лежит в скобках []()
- Дропнула дупликаты по описанию
- Предложения длиной меньше 5 символов удалены
- replace('\n', '').replace('\r', '').replace('\xad', ''))
'''